In [2]:
from keras.models import Sequential
from keras.layers import Dense

# import librart
import sklearn
import numpy as np
import pandas as pd
import pickle
from scipy.sparse import hstack
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score
import scipy
from scipy.sparse import csr_matrix
from sklearn.feature_selection import SelectKBest, mutual_info_classif
import feature_select
import feture_process
from scipy.sparse import load_npz
from scipy.sparse import load_npz
from sklearn.ensemble import BaggingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import train_test_split
from keras.callbacks import EarlyStopping
from sklearn.model_selection import GridSearchCV
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
import tensorflow

In [3]:
train_df = pd.read_csv("./project_data_files/book_rating_train.csv")
test_df = pd.read_csv("./project_data_files/book_rating_test.csv")

One hot code Processing

In [4]:
# process Prublisher and Language
train_df["Language"], test_df["Language"] = feture_process.docclass_preprocess(train_df["Language"],test_df["Language"],10)
train_df["Publisher"], test_df["Publisher"] = feture_process.docclass_preprocess(train_df["Publisher"],test_df["Publisher"],200)

In [5]:
# use oneHotCode for Publisher (sklearn)
publisher_train_hot = feture_process.process_OneHotEncoder(train_df,"Publisher")
publisher_test_hot = feture_process.process_OneHotEncoder(test_df,"Publisher")
train_df = pd.concat([train_df, publisher_train_hot], axis=1)
test_df = pd.concat([test_df, publisher_test_hot], axis=1)

In [6]:
# use oneHotCode for Language (sklearn)
language_train_hot = feture_process.process_OneHotEncoder(train_df,"Language")
language_test_hot = feture_process.process_OneHotEncoder(test_df,"Language")
train_df = pd.concat([train_df, language_train_hot], axis=1)
test_df = pd.concat([test_df, language_test_hot], axis=1)

In [7]:
# use oneHotCode for label (sklearn)
label_train_hot = feture_process.process_OneHotEncoder(train_df,"rating_label")
train_df = pd.concat([train_df, label_train_hot], axis=1)

Countvectorizer processing

In [8]:
# train_name_countvectorizer
train_name_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_name_countvectorizer.pkl", "rb"))
train_name_dic = train_name_countvectorizer.vocabulary_

# train_authors_countvectorizer
train_authors_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_authors_countvectorizer.pkl", "rb"))
train_authors_dic = train_authors_countvectorizer.vocabulary_

# train_desc_countvectorizer
train_desc_countvectorizer = pickle.load(open("./project_data_files/book_text_features_countvec/train_desc_countvectorizer.pkl", "rb"))
train_desc__dic = train_desc_countvectorizer.vocabulary_

# process vector features
train_name_features = train_name_countvectorizer.transform(train_df['Name'])
train_authors_features = train_authors_countvectorizer.transform(train_df['Authors'])
train_desc_features = train_desc_countvectorizer.transform(train_df['Description'])
other_features_df = train_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language','rating_label_3.0', 'rating_label_4.0', 'rating_label_5.0', 'rating_label'])

In [9]:
# new sparse features
sparse_features = hstack([train_name_features, train_authors_features, train_desc_features])
# new train features
dense_features = csr_matrix(other_features_df.values)
train_features = hstack([sparse_features, dense_features])


# process test features
test_name_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_name_vec.npz')
test_authors_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_authors_vec.npz')
test_desc_features = scipy.sparse.load_npz('./project_data_files/book_text_features_countvec/test_desc_vec.npz')
test_other_features_df = test_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])

test_sparse_features = hstack([test_name_features, test_authors_features, test_desc_features])
new_dense_features = csr_matrix(test_other_features_df.values)
test_features = hstack([test_sparse_features, new_dense_features])

doc2vec processing

In [ ]:
# use doc2vec
# use doc2vec
# use doc2vec
# use doc2vec

# process vector features
train_name_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_name_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
train_authors_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_authors_doc2vec20.csv", index_col = False, delimiter = ',', header=None)
train_desc_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/train_desc_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
other_features_df = train_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language', 'rating_label', 'rating_label_3.0', 'rating_label_4.0', 'rating_label_5.0'])
train_features = pd.concat([train_name_features, train_authors_features, train_desc_features, other_features_df], axis=1)

In [ ]:
# use doc2vec
# use doc2vec
# use doc2vec
# use doc2vec

# process test features
test_name_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_name_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
test_authors_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_authors_doc2vec20.csv", index_col = False, delimiter = ',', header=None)
test_desc_features = pd.read_csv(r"./project_data_files/book_text_features_doc2vec/test_desc_doc2vec100.csv", index_col = False, delimiter = ',', header=None)
test_other_features_df = test_df.drop(columns=['Name', 'Authors', 'Description', 'Publisher', 'Language'])
test_features = pd.concat([test_name_features, test_authors_features, test_desc_features, test_other_features_df], axis=1)


Feature selection

In [ ]:
selected_features,selected_features_test = train_features,test_features

In [ ]:
selected_features,selected_features_test = feature_select.MI(train_df,train_features,test_features,12000)

In [10]:
selected_features,selected_features_test = feature_select.chi_square(train_df,train_features,test_features,5000)

In [12]:
selected_features,selected_features_test = feature_select.variance_ratio(train_df,train_features,test_features,0.8)

In [ ]:
selected_features = load_npz("selected_features_label_one_hot_12000.npz")
selected_features_test = load_npz("selected_features_test_label_one_hot_12000.npz")

hyper parameter choose

In [ ]:
def create_mlp(input_dim, hidden_layers=1, hidden_units=64, activation='relu'):
    model = Sequential()
    model.add(Dense(hidden_units, input_dim=input_dim, activation=activation,kernel_regularizer=l2(0.0001)))

    for _ in range(hidden_layers - 1):
        model.add(Dense(hidden_units, activation=activation))

    model.add(Dense(3, activation='softmax'))
    model.compile(loss='sparse_categorical_crossentropy', optimizer='adam', metrics=['accuracy'])
    return model

In [ ]:
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
input_dim = selected_features.shape[1] 
mlp = KerasClassifier(build_fn=create_mlp, input_dim=input_dim, verbose=0)

param_grid = {
    'hidden_layers': [1, 2, 3],
    'hidden_units': [32, 64, 128],
    'activation': ['relu', 'tanh'],
    'batch_size': [32, 64],
    'epochs': [50, 100],
}
pipeline = Pipeline([
    ('scaler', StandardScaler()), 
    ('gridsearch', GridSearchCV(mlp, param_grid, cv=3, n_jobs=-1, verbose=1)),
])
X_train, X_val, y_train, y_val = train_test_split(selected_features, train_df["rating_label"], test_size=0.3, random_state=66)
X_train = X_train.toarray()
X_val = X_val.toarray()

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
best_params = pipeline.named_steps['gridsearch'].best_params_

train model

In [ ]:
from keras.regularizers import l1, l2
X_train, X_val, y_train, y_val = train_test_split(selected_features, label_train_hot, test_size=0.3, random_state=66)

input_dim = selected_features.shape[1] 
num_classes = label_train_hot.shape[1]

model = Sequential()
model.add(Dense(256, activation='leaky_relu', input_shape=(input_dim,)
                , kernel_regularizer=l2(0.0001)
                ))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))

model.add(Dense(num_classes, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])


In [ ]:
X_train = X_train.toarray()
X_val = X_val.toarray()

In [ ]:
class_weights = {
    (1, 0, 0): 2.0,
    (0, 1, 0): 1.0,
    (0, 0, 1): 3.0
}

In [ ]:
y_train_array = np.array(y_train)
sample_weights = np.array([class_weights[tuple(y)] for y in y_train_array])

In [ ]:
from sklearn.utils import class_weight
unique_classes = np.unique(y_train)
sample_weights = class_weight.compute_sample_weight(class_weight='balanced', y=y_train)

In [ ]:
mlp = model.fit(X_train, y_train, epochs=50, batch_size=32, validation_data=(X_val, y_val)
                , sample_weight=sample_weights
                )

In [ ]:
import matplotlib.pyplot as plt

plt.figure()
plt.plot(mlp.history['loss'], label='Training Loss')
plt.plot(mlp.history['val_loss'], label='Validation Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.title('Loss Learning Curve')

plt.figure()
plt.plot(mlp.history['accuracy'], label='Training Accuracy')
plt.plot(mlp.history['val_accuracy'], label='Validation Accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.title('Accuracy Learning Curve')

plt.show()

In [ ]:
type(selected_features)

In [ ]:
# for variance_ratio slection
import tensorflow as tf
coo_matrix = selected_features_test.tocoo()
indices = np.column_stack((coo_matrix.row, coo_matrix.col))
values = coo_matrix.data
shape = coo_matrix.shape

sparse_tensor = tf.SparseTensor(indices=indices, values=values, dense_shape=shape)
ordered_sparse_tensor = tf.sparse.reorder(sparse_tensor)

In [ ]:
prediction_prob = model.predict(ordered_sparse_tensor)

In [ ]:
# get predictions
prediction_index = np.argmax(prediction_prob, axis=1)
ratings = np.array([3.0, 4.0, 5.0])
prediction = ratings[prediction_index]

In [ ]:
output_df = pd.DataFrame({'rating_label': prediction})
output_df.index += 1
output_df.index.name = 'id'
output_df.to_csv('./predictions.csv')

MLP Bagging

In [13]:
import numpy as np
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.ensemble import BaggingClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score




Accuracy: 0.76


In [14]:
X_train, X_test, y_train, y_test = train_test_split(selected_features, train_df["rating_label"], test_size=0.3, random_state=42)


mlp = MLPClassifier(hidden_layer_sizes=(100,), activation='relu', solver='adam', alpha=0.0001, 
                    batch_size='auto', learning_rate='constant', learning_rate_init=0.001, 
                    max_iter=200, shuffle=True, random_state=None)


bagging = BaggingClassifier(base_estimator=mlp, n_estimators=10, max_samples=0.8, max_features=1.0, 
                            bootstrap=True, bootstrap_features=False, n_jobs=-1, random_state=42)


bagging.fit(X_train, y_train)

y_pred = bagging.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print(f"Accuracy: {accuracy:.2f}")

Accuracy: 0.76


In [18]:
y_pred = bagging.predict(selected_features_test)
output_df = pd.DataFrame({'rating_label': y_pred})
output_df.index += 1
output_df.index.name = 'id'
output_df.to_csv('./predictions.csv')